<a href="https://colab.research.google.com/github/S-EGK/State-Estimators/blob/main/EFK%20estimator%20for%20Quadrotor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
# Importing Libraries
import numpy as np
import scipy.signal as signal
import scipy.linalg as linalg
import math
from math import cos, sin, tan, sqrt
from scipy.integrate import solve_ivp
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from matplotlib.patches import Circle, PathPatch
import mpl_toolkits.mplot3d.art3d as art3d
import random

# Initialization

In [ ]:
# Parameters
g = 9.806650  #gravity
m = 1.56  #mass
Jx = 0.114700 #moi wrt x
Jy = 0.057600 #moi wrt y
Jz = 0.171200 #moi wrt z
Ts = 0.05 # sampling time

'States'

# coordinates
pn = 0 #north
pe = 0 #east
pd = 0 #down

# velocities
u = 0 #wrt north
v = 0 #wrt east
w = 0 #wrt down

# angles
phi = 0   #roll
theta = 0 #pitch
psi = 0   #yaw

# angular rates
p = 0 #wrt roll
q = 0 #wrt pitch
r = 0 #wrt yaw

# initial states
states = np.array([pn, pe, pd, u, v, w, phi, theta, psi, p, q, r])

# Data collection for trajectories and plots
pn_data = []
pe_data = []
h_data = []
vn_data = []
ve_data = []
vd_data = []
phi_data = []
theta_data = []
psi_data = []
p_data = []
q_data = []
r_data = []
phi_c_data = []
theta_c_data = []
r_c_data = []
pn_t_data = []
pe_t_data = []
pd_t_data = []
vn_t_data = []
ve_t_data = []
vd_t_data = []
psi_t_data = []

p_sensor_data = []
q_sensor_data = []
r_sensor_data = []
az_sensor_data = []
pn_sensor_data = []
pe_sensor_data = []
pd_sensor_data = []

#Simulation Parameters
Ts = 0.05 # sampling time
Tfinal = 15
Tsteps = Tfinal/Ts # number of frames
frames = int(Tsteps)

## Prediction Initialization

In [ ]:
# State Initialization
pn_hat = 0
pe_hat = 0
pd_hat = 0
u_hat = 0
v_hat = 0
w_hat = 0
psi_hat = 0
states_hat = np.array([[pn_hat, pe_hat, pd_hat, u_hat, v_hat, w_hat, psi_hat]], dtype = 'float').T

# Covariance Matrix
P_EKF = np.identity(7)

# Step Time
t_out = 0.05

# Covariance Matrices
Q_EKF = np.array([[0.1,0,0,0,0,0,0],
                  [0,0.1,0,0,0,0,0],
                  [0,0,0.1,0,0,0,0],
                  [0,0,0,0.1,0,0,0],
                  [0,0,0,0,0.1,0,0],
                  [0,0,0,0,0,0.1,0],
                  [0,0,0,0,0,0,0.1]])

R_EKF = np.array([[0.1]])

# Landmark coordinates
NL = 5
pn_l = np.array([-2,3,-4,1,2])
pe_l = np.array([-2,4,2,-3,-1])
pd_l = np.array([0,1,-4,1,-2])

# Data Collection
pn_hat_data = []
pe_hat_data = []
pd_hat_data = []
u_hat_data = []
v_hat_data = []
w_hat_data = []
psi_hat_data = []

P_EKF_pn_data=[]
P_EKF_pe_data=[]
P_EKF_pd_data=[]
P_EKF_u_data=[]
P_EKF_v_data=[]
P_EKF_w_data=[]
P_EKF_psi_data=[]

NP_EKF_pn_data=[]
NP_EKF_pe_data=[]
NP_EKF_pd_data=[]
NP_EKF_u_data=[]
NP_EKF_v_data=[]
NP_EKF_w_data=[]
NP_EKF_psi_data=[]

error_pn_hat = []
error_pe_hat = []
error_pd_hat = []
error_u_hat = []
error_v_hat = []
error_w_hat = []
error_psi_hat = []

# Rotation Matrices

In [ ]:
# Rotational Matrix to convert body frame to inertial frame
def rot_mat_bod2veh(phi, theta, psi):
    R_b_v=np.array([[cos(theta)*cos(psi), sin(phi)*sin(theta)*cos(psi)-cos(phi)*sin(psi), cos(phi)*sin(theta)*cos(psi)+sin(phi)*sin(psi)],
                    [cos(theta)*sin(psi), sin(phi)*sin(theta)*sin(psi)+cos(phi)*cos(psi), cos(phi)*sin(theta)*sin(psi)-sin(phi)*cos(psi)],
                    [-sin(theta),         sin(phi)*cos(theta),                            cos(phi)*cos(theta)]])
    return R_b_v

In [ ]:
# Matrix to orient ned to xyz
def axes_ned2yxz():
  A_ned = [[0, 1, 0],
           [1, 0, 0],
           [0, 0, -1]]
  return A_ned

In [ ]:
# Matrix with relationship between derivatives of angles to angular rates
def der_ang2rates(phi, theta):
  sPh = np.sin(phi)
  cPh = np.cos(phi)
  sT = np.sin(theta)
  cT = np.cos(theta)
  TT = np.tan(theta)
  d_a_r = [[1, sPh*TT, cPh*TT],
           [0, cPh,    -sPh],
           [0, sPh/cT, cPh/cT]]
  return d_a_r

In [ ]:
# Rotation matrix to eliminate psi
def psi_rot_mat(psi):
  cPs = np.cos(psi)
  sPs = np.sin(psi)
  R_psi = np.array([[cPs, sPs, 0],
                    [-sPs, cPs, 0],
                    [0, 0, 1]])
  return R_psi

# Controllers

## Attitude Controller

In [ ]:
# A, B, Q, R, K matrix for attitude controller
A = np.array([[0, 1, 0, 0, 0],
              [0, 0, 0, 0, 0],
              [0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0]])

B = np.array([[0, 0, 0],
              [1/Jx, 0, 0],
              [0, 0, 0],
              [0, 1/Jy, 0],
              [0, 0, 1/Jz]])

Q = np.array([[10, 0, 0, 0, 0],
              [0, 0.1, 0, 0, 0],
              [0, 0, 10, 0, 0],
              [0, 0, 0, 0.1, 0],
              [0, 0, 0, 0, 1]])

R1 = [[0.1, 0, 0],
      [0, 0.1, 0],
      [0, 0, 0.1]]

P = np.matrix(linalg.solve_continuous_are(A, B, Q, R1))
K = np.matrix(linalg.inv(R1)*(B.T*P))

eigVal, eigVec = linalg.eig(A-B*K)

In [ ]:
# LQR controller for Attitude Control
def roll_lqr(y, y_c, K):
  u = -K[0,0]*(y[0]-y_c[0]) -K[0,1]*(y[1]-y_c[1]) -K[0,2]*(y[2]-y_c[2]) -K[0,3]*(y[3]-y_c[3]) -K[0,4]*(y[4]-y_c[4])
  return u

def pitch_lqr(y, y_c, K):
  u = -K[1,0]*(y[0]-y_c[0])-K[1,1]*(y[1]-y_c[1]) -K[1,2]*(y[2]-y_c[2]) -K[1,3]*(y[3]-y_c[3]) -K[1,4]*(y[4]-y_c[4])
  return u

def yaw_lqr(y, y_c, K):
  u = -K[2,0]*(y[0]-y_c[0]) -K[2,1]*(y[1]-y_c[1]) -K[2,2]*(y[2]-y_c[2]) -K[2,3]*(y[3]-y_c[3]) -K[2,4]*(y[4]-y_c[4])
  return u

## Trajectory Controller

In [ ]:
# A, B, Q, R, K matrix for trajectory controller
A1 = np.array([[0, 0, 0, 1, 0, 0, 0],
               [0, 0, 0, 0, 1, 0, 0],
               [0, 0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0]])

B1 = np.array([[0, 0, 0, 0],
               [0, 0, 0, 0],
               [0, 0, 0, 0],
               [1, 0, 0, 0],
               [0, 1, 0, 0],
               [0, 0, 1, 0],
               [0, 0, 0, 1]])

Q1 = np.array([[10**3, 0, 0, 0, 0, 0, 0],
               [0, 10**3, 0, 0, 0, 0, 0],
               [0, 0, 10**3, 0, 0, 0, 0],
               [0, 0, 0, 10**2, 0, 0, 0],
               [0, 0, 0, 0, 10**2, 0, 0],
               [0, 0, 0, 0, 0, 50, 0],
               [0, 0, 0, 0, 0, 0, 10]])

R2 = np.array([[25, 0, 0, 0],
               [0, 25, 0, 0],
               [0, 0, 10, 0],
               [0, 0, 0, 10]])

P1 = np.matrix(linalg.solve_continuous_are(A1, B1, Q1, R2))
K1 = np.matrix(linalg.inv(R2)*(B1.T*P1))

eigVal1, eigVec1 = linalg.eig(A1-B1*K1)

In [ ]:
# Function to generate reference trajectory and differential flatness
def trajectory (t):
  # #case a
  # a=1.5
  # b=0.75
  # c=0
  # n=-0.75
  # T=5
  # w1=2*np.pi/T
  # w2=w1/2
  # w3=w1

  # #case b
  # a=1.5
  # b=0.75
  # c=0.5
  # n=-0.75
  # T=10
  # w1=2*np.pi/T
  # w2=w1/2
  # w3=w1
  
  #case c
  a=0.75
  b=0.75
  c=0
  n=-0.75
  T=10
  w1=2*np.pi/T
  w2=w1
  w3=w1

  # #case d
  # a=1.5
  # b=a/2
  # c=b/2
  # n=-0.75
  # T=5*2
  # w1=2*np.pi/T
  # w2=w1/2
  # w3=w1/3
   
  ytraj=np.array([[a*cos(w2*t)], [b*sin(w1*t)], [n+c*sin(w3*t)], [np.pi/4]])
  ydottraj=np.array([[-a*w2*sin(w2*t)], [b*w1*cos(w1*t)], [c*w3*cos(w3*t)], [0]])
  yddottraj=np.array([[-a*w2*w2*cos(w2*t)], [-b*w1*w1*sin(w1*t)], [-c*w3*w3*sin(w3*t)], [0]])

  # ytraj=np.array([[a*signal.square(w2*t)], [b*signal.square(w1*t)], [n+c*sin(w3*t)], [np.pi/4]])
  # ydottraj=np.array([[-a*w2*sin(w2*t)], [b*w1*cos(w1*t)], [c*w3*cos(w3*t)], [0]])
  # yddottraj=np.array([[-a*w2*w2*cos(w2*t)], [-b*w1*w1*sin(w1*t)], [-c*w3*w3*sin(w3*t)], [0]])

  ur=np.array([yddottraj[0], yddottraj[1], yddottraj[2]-g, yddottraj[3]])   
  xr=np.array([ytraj[0], ytraj[1], ytraj[2], ydottraj[0], ydottraj[1], ydottraj[2], ytraj[3]])

  pn_t_data.append(a*cos(w2*t))
  pe_t_data.append(b*sin(w1*t))
  pd_t_data.append(-(n+c*sin(w3*t)))
  vn_t_data.append(-a*w2*sin(w2*t))
  ve_t_data.append(b*w1*cos(w1*t))
  vd_t_data.append(-c*w3*w3*sin(w3*t))
  psi_t_data.append(np.pi/4)

  return ur, xr

In [ ]:
# LQR and Inverse mapping for trajectory control
def trajectory_control(states, xr, ur):
  pn = states[0]
  pe = states[1]
  pd = states[2]
  u = states[3]
  v = states[4]
  w = states[5]
  phi = states[6]
  theta = states[7]
  psi = states[8]
  p = states[9]
  q = states[10]
  r = states[11]

  x=np.array([[pn], [pe], [pd], [u], [v], [w], [psi]]) # 7 state system

  dx = x-xr # error from true to desired states
  du = -np.matmul(K1,dx)  # error input = -K*x (LQR)
  uc = du+ur  # reference input + error of input

  up = np.array([uc[0,0], uc[1,0], uc[2,0]])

  F = m*np.sqrt(np.matmul(up.transpose(), up))  # Thrust to input into dynamics

  R_psi = psi_rot_mat(psi)
  z = -np.matmul(R_psi, up)*m/F

  phi_c = np.arcsin(-z[1])  # commanded phi
  theta_c = np.arctan2(z[0], z[2])  # commanded theta
  r_c = uc[3,0]*cos(phi)*cos(theta)-q*sin(phi)  # commanded r

  return F, float(phi_c), float(theta_c), float(r_c)

# Sensor Models

## Gyro Sensor

In [ ]:
# Gyro Sensor Model
def gyro(p,q,r):
  gyro = np.array([0, 0, 0], dtype = 'float')
  gyro[0] = p + 0.0001 * random.random()
  gyro[1] = q + 0.0001 * random.random()
  gyro[2] = r + 0.0001 * random.random()
  return gyro

## Accelerometer Sensor

In [ ]:
# Acceloerometer Sensor model
def accm(F):
  accm = -F/m + 0.0001 * random.random()
  return accm

## Range Sensor

In [ ]:
# Range Sensor Model
def range1(pn, pe, pd):
  rho = []
  for i in range(NL):
    rho.append(np.sqrt((pn_l[i]-pn)**2+(pe_l[i]-pe)**2+(pd_l[i]-pd)**2) + 0.03 * random.random())
  return rho

# Quad Drawing

## Quad Build Function

In [ ]:
# function to draw the quad
def build_quad(pn, pe, pd, phi, theta, psi):
  pn = pn
  pe = pe
  pd = pd
  phi = phi
  theta = theta
  psi = psi
  
  # define width to use as reference to construct the quad 
  w = 0.75

  # defining the coordinates of the end points of the quad rotors
  p1 = np.array([w, 0, 0]).T
  p2 = np.array([-w, 0, 0]).T
  p3 = np.array([0, w, 0]).T
  p4 = np.array([0, -w, 0]).T

  # converting the quadrotor points to interial frame and translating as per ned
  R = rot_mat_bod2veh(phi, theta, psi) # rotational matrix
  pos_ned = np.array([pn, pe, pd]).T #
  p1r = np.matmul(R,p1)+pos_ned
  p2r = np.matmul(R,p2)+pos_ned
  p3r = np.matmul(R,p3)+pos_ned
  p4r = np.matmul(R,p4)+pos_ned

  # changing the orientation of ned wrt yxz
  A = axes_ned2yxz()
  p1ra = np.matmul(A,p1r)
  p2ra = np.matmul(A,p2r)
  p3ra = np.matmul(A,p3r)
  p4ra = np.matmul(A,p4r)

  # Rotor coordinates
  rot_x = [p1ra[0], p2ra[0], p3ra[0], p4ra[0]]
  rot_y = [p1ra[1], p2ra[1], p3ra[1], p4ra[1]]
  rot_z = [p1ra[2], p2ra[2], p3ra[2], p4ra[2]]

  # Arm coordinates
  arm1_x = [p1ra[0], p2ra[0]]
  arm1_y = [p1ra[1], p2ra[1]]
  arm1_z = [p1ra[2], p2ra[2]]

  arm2_x = [p3ra[0], p4ra[0]]
  arm2_y = [p3ra[1], p4ra[1]]
  arm2_z = [p3ra[2], p4ra[2]]

  # defining the coordinates of the payload
  v1 = np.array([-w/3,-w/3,w/2]).T
  v2 = np.array([-w/3,-w/3,0]).T
  v3 = np.array([w/3,-w/3,0]).T
  v4 = np.array([w/3,-w/3,w/2]).T
  v5 = np.array([w/3,w/3,w/2]).T
  v6 = np.array([w/3,w/3,0]).T
  v7 = np.array([-w/3,w/3,0]).T
  v8 = np.array([-w/3,w/3,w/2]).T

  # converting the payload points to interial frame and translating as per ned
  v1r = np.matmul(R,v1)+pos_ned
  v2r = np.matmul(R,v2)+pos_ned
  v3r = np.matmul(R,v3)+pos_ned
  v4r = np.matmul(R,v4)+pos_ned
  v5r = np.matmul(R,v5)+pos_ned
  v6r = np.matmul(R,v6)+pos_ned
  v7r = np.matmul(R,v7)+pos_ned
  v8r = np.matmul(R,v8)+pos_ned

  # changing the orientation of ned wrt yxz
  v1ra = np.matmul(A,v1r)
  v2ra = np.matmul(A,v2r)
  v3ra = np.matmul(A,v3r)
  v4ra = np.matmul(A,v4r)
  v5ra = np.matmul(A,v5r)
  v6ra = np.matmul(A,v6r)
  v7ra = np.matmul(A,v7r)
  v8ra = np.matmul(A,v8r)

  # defining the faces of the payload
  f1_x=[v1ra[0], v2ra[0], v3ra[0], v4ra[0], v1ra[0]]
  f1_y=[v1ra[1], v2ra[1], v3ra[1], v4ra[1], v1ra[1]]
  f1_z=[v1ra[2], v2ra[2], v3ra[2], v4ra[2], v1ra[2]]
  f2_x=[v5ra[0], v6ra[0], v7ra[0], v8ra[0], v5ra[0]]
  f2_y=[v5ra[1], v6ra[1], v7ra[1], v8ra[1], v5ra[1]]
  f2_z=[v5ra[2], v6ra[2], v7ra[2], v8ra[2], v5ra[2]]
  f3_x=[v4ra[0], v5ra[0], v6ra[0], v3ra[0], v4ra[0]]
  f3_y=[v4ra[1], v5ra[1], v6ra[1], v3ra[1], v4ra[1]]
  f3_z=[v4ra[2], v5ra[2], v6ra[2], v3ra[2], v4ra[2]]
  f4_x=[v1ra[0], v8ra[0], v7ra[0], v2ra[0], v1ra[0]]
  f4_y=[v1ra[1], v8ra[1], v7ra[1], v2ra[1], v1ra[1]]
  f4_z=[v1ra[2], v8ra[2], v7ra[2], v2ra[2], v1ra[2]]

  return rot_x, rot_y, rot_z, arm1_x, arm1_y, arm1_z, arm2_x, arm2_y, arm2_z, f1_x, f1_y, f1_z, f2_x, f2_y, f2_z, f3_x, f3_y, f3_z, f4_x, f4_y, f4_z

## Initial Plot

In [ ]:
# Intial Plot
fig = plt.figure(figsize=(7,7), dpi=90)
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.set_xlim((-3, 3))            
ax.set_ylim((-3, 3))
ax.set_zlim((-3, 3))
ax.set_xlabel('East(x)')
ax.set_ylabel('North(y)')
ax.set_zlabel('Down(z)')
ax.set_title('Quadrotor Simulation (y:ref, k:true)')

rot_x, rot_y, rot_z, arm1_x, arm1_y, arm1_z, arm2_x, arm2_y, arm2_z, f1_x, f1_y, f1_z, f2_x, f2_y, f2_z, f3_x, f3_y, f3_z, f4_x, f4_y, f4_z = build_quad(pn, pe, pd, phi, theta, psi)
rot_x_hat, rot_y_hat, rot_z_hat, arm1_x_hat, arm1_y_hat, arm1_z_hat, arm2_x_hat, arm2_y_hat, arm2_z_hat, f1_x_hat, f1_y_hat, f1_z_hat, f2_x_hat, f2_y_hat, f2_z_hat, f3_x_hat, f3_y_hat, f3_z_hat, f4_x_hat, f4_y_hat, f4_z_hat = build_quad(pn_hat, pe_hat, pd_hat, phi, theta, psi_hat)

# payload
face1, = ax.plot(f1_x, f1_y, f1_z, 'b', lw=1.1)
face2, = ax.plot(f2_x, f2_y, f2_z, 'r', lw=1.1)
face3, = ax.plot(f3_x, f3_y, f3_z, 'g', lw=1.1)
face4, = ax.plot(f4_x, f4_y, f4_z, 'y', lw=1.1)

# arms and rotors
arm1, = ax.plot(arm1_x, arm1_y, arm1_z, 'r-', lw=1.5)
arm2, = ax.plot(arm2_x, arm2_y, arm2_z, 'k-', lw=1.5)
rotor, = ax.plot(rot_x, rot_y, rot_z, 'bo', ms = 5)

#trajectory
traj, = ax.plot(pe_data, pn_data, h_data, 'k:', lw=1)
traj_ref, = ax.plot(pe_t_data, pn_t_data, pd_t_data, 'y:', lw=1)

# Estimated Quad
# payload
face1_hat, = ax.plot(f1_x_hat, f1_y_hat, f1_z_hat, 'b:', lw=1.1)
face2_hat, = ax.plot(f2_x_hat, f2_y_hat, f2_z_hat, 'r:', lw=1.1)
face3_hat, = ax.plot(f3_x_hat, f3_y_hat, f3_z_hat, 'g:', lw=1.1)
face4_hat, = ax.plot(f4_x_hat, f4_y_hat, f4_z_hat, 'y:', lw=1.1)

# arms and rotors
arm1_hat, = ax.plot(arm1_x_hat, arm1_y_hat, arm1_z_hat, 'r:', lw=1.5)
arm2_hat, = ax.plot(arm2_x_hat, arm2_y_hat, arm2_z_hat, 'k:', lw=1.5)
rotor_hat, = ax.plot(rot_x_hat, rot_y_hat, rot_z_hat, 'bo', ms = 5)

#trajectory
traj_hat, = ax.plot(pe_hat_data, pn_hat_data, pd_hat_data, 'm:', lw=1)

# True States

## Dynamics

In [ ]:
# Dynamics of the Quadrotor
def quad_dyna(t, y, F, T_phi, T_theta, T_psi):
  pn = y[0]
  pe = y[1]
  pd = y[2]
  u = y[3]
  v = y[4]
  w = y[5]
  phi = y[6]
  theta = y[7]
  psi = y[8]
  p = y[9]
  q = y[10]
  r = y[11]

  # x_dot of pned, ie velocities
  pn_dot = u
  pe_dot = v
  pd_dot = w
  
  # vel_dot, ie accelerations 
  R_b_w = rot_mat_bod2veh(phi,theta,psi)
  Thrust = np.array([0, 0, -F/m]).T
  gravity = np.array([0, 0, g]).T
  Accn = np.matmul(R_b_w, Thrust) + gravity
  
  u_dot = Accn[0]
  v_dot = Accn[1]
  w_dot = Accn[2]

  # angle_dot, ie angular rates
  Der = der_ang2rates(phi, theta)
  Ang_vel = np.matmul(Der, np.array([p,q,r]).T)
  
  phi_dot = Ang_vel[0]
  theta_dot = Ang_vel[1]
  psi_dot = Ang_vel[2]
  
  # angular rates_dot, ie angular accelerations
  Ang_accn = np.array([T_phi/Jx,T_theta/Jy,T_psi/Jz]).T + np.array([q*r*(Jy-Jz)/Jx,p*r*(Jz-Jx)/Jy,q*p*(Jx-Jy)/Jz]).T

  p_dot = Ang_accn[0]
  q_dot = Ang_accn[1]
  r_dot = Ang_accn[2]

  return pn_dot, pe_dot, pd_dot, u_dot, v_dot, w_dot, phi_dot, theta_dot, psi_dot, p_dot, q_dot, r_dot

## Solver

In [ ]:
# Solving the state space equation
def step(states, Ts, i):
  pn=states[0]
  pe=states[1]
  pd=states[2]
  u=states[3]
  v=states[4]
  w=states[5]
  phi=states[6]
  theta=states[7]
  psi=states[8]
  p=states[9]
  q=states[10]
  r=states[11]

  ur, xr = trajectory(i/15) # trajectory + diff flat # i/15 is taken to get a smooth curve

  F, phi_c, theta_c, r_c = trajectory_control(states, xr, ur)  # LQR + inv map

  y = np.array([phi, p, theta, q, r]) # states for attitude controller
  y_c = np.array([phi_c, 0, theta_c, 0, r_c]) # commanded states for attitude controller

  # Angular Inputs from LQR
  T_phi = roll_lqr(y, y_c, K)
  T_theta = pitch_lqr(y, y_c, K)
  T_psi = yaw_lqr(y, y_c, K)

  sol = solve_ivp(lambda t, y: quad_dyna(t, y, F, T_phi, T_theta, T_psi), [0, Ts], [pn, pe, pd, u, v, w, phi, theta, psi, p, q, r], t_eval=np.linspace(0, Ts, 10))
  states=sol.y[:,-1].T

  # Data collection for plots
  pn_data.append(pn)
  pe_data.append(pe)
  h_data.append(-pd)
  vn_data.append(u)
  ve_data.append(v)
  vd_data.append(w)
  phi_data.append(phi)
  theta_data.append(theta)
  psi_data.append(psi)
  p_data.append(p)
  q_data.append(q)
  r_data.append(r)
  phi_c_data.append(phi_c)
  theta_c_data.append(theta_c)
  r_c_data.append(r_c)

  return states, F

# Animation

In [ ]:
def drawframe(i):
  global states
  global states_hat
  global P_EKF
  global Q_EKF
  global R_EKF

  [pn_hat] = states_hat[0]
  [pe_hat] = states_hat[1]
  [pd_hat] = states_hat[2]
  [u_hat] = states_hat[3]
  [v_hat] = states_hat[4]
  [w_hat] = states_hat[5]
  [psi_hat] = states_hat[6]
  
  # True States Solver
  states, F = step(states, Ts, i)
  pn = states[0]
  pe = states[1]
  pd = states[2]
  u = states[3]
  v = states[4]
  w = states[5]
  phi = states[6]
  theta = states[7]
  psi = states[8]
  p = states[9]
  q = states[10]
  r = states[11]

  # Dynamic model for prediction
  # Eqn 1
  pn_hat_dot = u_hat
  pe_hat_dot = v_hat
  pd_hat_dot = w_hat
  
  # Eqn 2
  Rbw = rot_mat_bod2veh(phi, theta, psi_hat) # rotational matrix
  az = accm(F)
  thrust = np.array([0, 0, az]).T
  gravity = np.array([0, 0, g]).T
  temp1 = np.matmul(Rbw, thrust) + gravity
  u_hat_dot = temp1[0]
  v_hat_dot = temp1[1]
  w_hat_dot = temp1[2]

  # Eqn 3
  temp2 = gyro(p, q, r)
  p_hat = temp2[0]
  q_hat = temp2[1]
  r_hat = temp2[2]
  psi_hat_dot = q_hat*(sin(phi)/cos(theta)) + r_hat*(cos(phi)/cos(theta))

  states_hat_dot = np.array([[pn_hat_dot],
                             [pe_hat_dot],
                             [pd_hat_dot],
                             [u_hat_dot],
                             [v_hat_dot],
                             [w_hat_dot],
                             [psi_hat_dot]])
  
  
  states_hat = states_hat + Ts * states_hat_dot
  [pn_hat] = states_hat[0] 
  [pe_hat] = states_hat[1]
  [pd_hat] = states_hat[2]
  [u_hat] = states_hat[3]
  [v_hat] = states_hat[4]
  [w_hat] = states_hat[5]
  [psi_hat] = states_hat[6]

  # jacobian for the state space equation
  A_jac = np.array([[0, 0, 0, 1, 0, 0, 0],
                    [0, 0, 0, 0, 1, 0, 0],
                    [0, 0, 0, 0, 0, 1, 0],
                    [0, 0, 0, 0, 0, 0, az*cos(psi_hat)*sin(phi) - az*cos(phi)*sin(psi_hat)*sin(theta)],
                    [0, 0, 0, 0, 0, 0, az*sin(phi)*sin(psi_hat) + az*cos(phi)*cos(psi_hat)*sin(theta)],
                    [0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0]])
  
  P_EKF_dot = np.matmul(A_jac, P_EKF) + np.matmul(P_EKF, A_jac.transpose()) + Q_EKF
  P_EKF = P_EKF + Ts*P_EKF_dot

  rho = range1(pn, pe, pd)
  
  #Update
  for i in range(NL):
    C_i = np.array([[(pn_hat-pn_l[i])/(math.sqrt((pn_hat-pn_l[i])**2+(pe_hat-pe_l[i])**2+(pd_hat-pd_l[i])**2)),(pe_hat-pe_l[i])/(math.sqrt((pn_hat-pn_l[i])**2+(pe_hat-pe_l[i])**2+(pd_hat-pd_l[i])**2)),(pd_hat-pd_l[i])/(math.sqrt((pn_hat-pn_l[i])**2+(pe_hat-pe_l[i])**2+(pd_hat-pd_l[i])**2)),0,0,0,0]])
    var1 = np.matmul(C_i, P_EKF)
    var2 = np.matmul(var1, np.transpose(C_i))
    var3 = np.linalg.inv(np.array(R_EKF+var2))
    var4 = np.matmul(np.transpose(C_i), var3)
    L_i = np.matmul(P_EKF, var4)
    var5 = rho[i]-math.sqrt((pn_hat-pn_l[i])**2+(pe_hat-pe_l[i])**2+(pd_hat-pd_l[i])**2)
    states_hat = states_hat + var5*L_i
    [pn_hat] = states_hat[0]
    [pe_hat] = states_hat[1]
    [pd_hat] = states_hat[2]
    [u_hat] = states_hat[3]
    [v_hat] = states_hat[4]
    [w_hat] = states_hat[5]
    [psi_hat] = states_hat[6]
    var6 = np.matmul(L_i,C_i)
    var7 = np.identity(7)-var6
    P_EKF = np.matmul(var7, P_EKF)

  pn_hat_data.append(pn_hat)
  pe_hat_data.append(pe_hat)
  pd_hat_data.append(-pd_hat)
  u_hat_data.append(u_hat)
  v_hat_data.append(v_hat)
  w_hat_data.append(w_hat)
  psi_hat_data.append(psi_hat)

  P_EKF_pn_data.append(3*math.sqrt(P_EKF[0,0]))
  P_EKF_pe_data.append(3*math.sqrt(P_EKF[1,1]))
  P_EKF_pd_data.append(3*math.sqrt(P_EKF[2,2]))
  P_EKF_u_data.append(3*math.sqrt(P_EKF[3,3]))
  P_EKF_v_data.append(3*math.sqrt(P_EKF[4,4]))
  P_EKF_w_data.append(3*math.sqrt(P_EKF[5,5]))
  P_EKF_psi_data.append(3*math.sqrt(P_EKF[6,6]))

  NP_EKF_pn_data.append(-3*math.sqrt(P_EKF[0,0]))
  NP_EKF_pe_data.append(-3*math.sqrt(P_EKF[1,1]))
  NP_EKF_pd_data.append(-3*math.sqrt(P_EKF[2,2]))
  NP_EKF_u_data.append(-3*math.sqrt(P_EKF[3,3]))
  NP_EKF_v_data.append(-3*math.sqrt(P_EKF[4,4]))
  NP_EKF_w_data.append(-3*math.sqrt(P_EKF[5,5]))
  NP_EKF_psi_data.append(-3*math.sqrt(P_EKF[6,6]))

  error_pn_hat.append(pn-pn_hat)
  error_pe_hat.append(pe-pe_hat)
  error_pd_hat.append(pd-pd_hat)
  error_u_hat.append(u-u_hat)
  error_v_hat.append(v-v_hat)
  error_w_hat.append(w-w_hat)
  error_psi_hat.append(psi-psi_hat)

  # Build quad using true states and estimated states
  rot_x, rot_y, rot_z, arm1_x, arm1_y, arm1_z, arm2_x, arm2_y, arm2_z, f1_x, f1_y, f1_z, f2_x, f2_y, f2_z, f3_x, f3_y, f3_z, f4_x, f4_y, f4_z = build_quad(pn, pe, pd, phi, theta, psi)
  rot_x_hat, rot_y_hat, rot_z_hat, arm1_x_hat, arm1_y_hat, arm1_z_hat, arm2_x_hat, arm2_y_hat, arm2_z_hat, f1_x_hat, f1_y_hat, f1_z_hat, f2_x_hat, f2_y_hat, f2_z_hat, f3_x_hat, f3_y_hat, f3_z_hat, f4_x_hat, f4_y_hat, f4_z_hat = build_quad(pn_hat, pe_hat, pd_hat, phi, theta, psi_hat)

  face1.set_data(f1_x, f1_y)
  face1.set_3d_properties(f1_z)
  face2.set_data(f2_x, f2_y)
  face2.set_3d_properties(f2_z)
  face3.set_data(f3_x, f3_y)
  face3.set_3d_properties(f3_z)
  face4.set_data(f4_x, f4_y)
  face4.set_3d_properties(f3_z)

  arm1.set_data(arm1_x, arm1_y)
  arm1.set_3d_properties(arm1_z)
  arm2.set_data(arm2_x, arm2_y)
  arm2.set_3d_properties(arm2_z)
  rotor.set_data(rot_x, rot_y)
  rotor.set_3d_properties(rot_z)

  traj.set_data(pe_data, pn_data)
  traj.set_3d_properties(h_data)

  traj_ref.set_data(pe_t_data, pn_t_data)
  traj_ref.set_3d_properties(pd_t_data)

  face1_hat.set_data(f1_x_hat, f1_y_hat)
  face1_hat.set_3d_properties(f1_z_hat)
  face2_hat.set_data(f2_x_hat, f2_y_hat)
  face2_hat.set_3d_properties(f2_z_hat)
  face3_hat.set_data(f3_x_hat, f3_y_hat)
  face3_hat.set_3d_properties(f3_z_hat)
  face4_hat.set_data(f4_x_hat, f4_y_hat)
  face4_hat.set_3d_properties(f3_z_hat)

  arm1_hat.set_data(arm1_x_hat, arm1_y_hat)
  arm1_hat.set_3d_properties(arm1_z_hat)
  arm2_hat.set_data(arm2_x_hat, arm2_y_hat)
  arm2_hat.set_3d_properties(arm2_z_hat)
  rotor_hat.set_data(rot_x_hat, rot_y_hat)
  rotor_hat.set_3d_properties(rot_z_hat)

  traj_hat.set_data(pe_hat_data, pn_hat_data)
  traj_hat.set_3d_properties(pd_hat_data)

  return face1, face2, face3, face4, arm1, arm2, rotor, traj, traj_ref, face1_hat, face2_hat, face3_hat, face4_hat, arm1_hat, arm2_hat, rotor_hat, traj_hat

In [ ]:
# Creating the Simulatiom
ax.set_xlabel('East(x)')
ax.set_ylabel('North(y)')
ax.set_zlabel('Down(z)')
ax.set_title('Quadrotor Simulation (y:ref, k:true, m:ekf)')
# blit=True re-draws only the parts that have changed.
anim = animation.FuncAnimation(fig, drawframe, frames=frames, interval=40, blit=True)

## Video

In [ ]:
# Converting the animated result into a html5 video for visualization
HTML(anim.to_html5_video())

# Plots

## State Plots

In [ ]:
# Figure to plot the states against time step
fig1 = plt.figure(figsize=(20,20))

ax1 = fig1.add_subplot(4, 3, 1)
ax1.set_xlabel('Time Step')
ax1.set_ylabel('Pn')
ax1.set_title('State Plot of North (b:true, r:ref, bl:ekf)')
pn1 = ax1.plot(range(len(pn_data)),pn_data)
pn_t1 = ax1.plot(range(len(pn_t_data)),pn_t_data, 'r:')
pn_hat1 = ax1.plot(range(len(pn_hat_data)),pn_hat_data, 'k--')

ax2 = fig1.add_subplot(4, 3, 2)
ax2.set_xlabel('Time Step')
ax2.set_ylabel('Pe')
ax2.set_title('State Plot of East (b:true, r:ref, bl:ekf)')
pe1 = ax2.plot(range(len(pe_data)),pe_data)
pe_t1 = ax2.plot(range(len(pe_t_data)),pe_t_data, 'r:')
pe_hat1 = ax2.plot(range(len(pe_hat_data)),pe_hat_data, 'k--')

ax3 = fig1.add_subplot(4, 3, 3)
ax3.set_xlabel('Time Step')
ax3.set_ylabel('h')
ax3.set_title('State Plot of Down (b:true, r:ref, bl:ekf)')
h1 = ax3.plot(range(len(h_data)),h_data)
pd_t1 = ax3.plot(range(len(pd_t_data)),pd_t_data, 'r:')
pd_hat1 = ax3.plot(range(len(pd_hat_data)),pd_hat_data, 'k--')

ax4 = fig1.add_subplot(4, 3, 4)
ax4.set_xlabel('Time Step')
ax4.set_ylabel('Vn')
ax4.set_title('State Plot of Velocity due north (b:true, r:ref, bl:ekf)')
vn1 = ax4.plot(range(len(vn_data)),vn_data)
vn_t1 = ax4.plot(range(len(vn_t_data)),vn_t_data, 'r:')
vn_hat1 = ax4.plot(range(len(u_hat_data)),u_hat_data, 'k--')

ax5 = fig1.add_subplot(4, 3, 5)
ax5.set_xlabel('Time Step')
ax5.set_ylabel('Ve')
ax5.set_title('State Plot of Velocity due east (b:true, r:ref, bl:ekf)')
ve1 = ax5.plot(range(len(ve_data)),ve_data)
ve_t1 = ax5.plot(range(len(ve_t_data)),ve_t_data, 'r:')
ve_hat1 = ax5.plot(range(len(v_hat_data)),v_hat_data, 'k--')

ax6 = fig1.add_subplot(4, 3, 6)
ax6.set_xlabel('Time Step')
ax6.set_ylabel('Vd')
ax6.set_title('State Plot of Velocity due down (b:true, r:ref, bl:ekf)')
vd1 = ax6.plot(range(len(vd_data)),vd_data)
vd_t1 = ax6.plot(range(len(vd_t_data)),vd_t_data, 'r:')
vd_hat1 = ax6.plot(range(len(w_hat_data)),w_hat_data, 'k--')

ax7 = fig1.add_subplot(4, 3, 7)
ax7.set_xlabel('Time Step')
ax7.set_ylabel('Phi')
ax7.set_title('State Plot of Phi (b:true, r:cmd)')
phi1 = ax7.plot(range(len(phi_data)),phi_data)
phi_c1 = ax7.plot(range(len(phi_c_data)),phi_c_data, 'r:')

ax8 = fig1.add_subplot(4, 3, 8)
ax8.set_xlabel('Time Step')
ax8.set_ylabel('Theta')
ax8.set_title('State Plot of Theta (b:true, r:cmd)')
theta1 = ax8.plot(range(len(theta_data)),theta_data)
theta_c1 = ax8.plot(range(len(theta_c_data)),theta_c_data, 'r:')

ax9 = fig1.add_subplot(4, 3, 9)
ax9.set_xlabel('Time Step')
ax9.set_ylabel('Psi')
ax9.set_title('State Plot of Psi (b:true, r:ref, bl:ekf)')
psi1 = ax9.plot(range(len(psi_data)),psi_data)
psi_t1 = ax9.plot(range(len(psi_t_data)),psi_t_data, 'r:')
psi_hat1 = ax9.plot(range(len(psi_hat_data)),psi_hat_data, 'k--')

ax10 = fig1.add_subplot(4, 3, 10)
ax10.set_xlabel('Time Step')
ax10.set_ylabel('P')
ax10.set_title('State Plot of Angular rate wrt Phi')
p1 = ax10.plot(range(len(p_data)),p_data)

ax11 = fig1.add_subplot(4, 3, 11)
ax11.set_xlabel('Time Step')
ax11.set_ylabel('Q')
ax11.set_title('State Plot of Angular rate wrt Theta')
q1 = ax11.plot(range(len(q_data)),q_data)

ax12 = fig1.add_subplot(4, 3, 12)
ax12.set_xlabel('Time Step')
ax12.set_ylabel('R')
ax12.set_title('State Plot of Angular rate wrt Psi (b:true, r:cmd)')
r1 = ax12.plot(range(len(r_data)),r_data)
r_c1 = ax12.plot(range(len(r_c_data)),r_c_data, 'r:')

## Error and Error Covariance

In [ ]:
# Plot show error between true states and estimated states using EKF
fig2 = plt.figure(figsize=(20,15))

ax13 = fig2.add_subplot(3, 3, 1)
ax13.set_xlabel('Time Step'), 
ax13.set_ylabel('Pn')
ax13.set_title('Error for Pn')
P_EKF_pn1 = ax13.plot(range(len(P_EKF_pn_data)),P_EKF_pn_data, 'b')
error_pn1 = ax13.plot(range(len(error_pn_hat)),error_pn_hat, 'r')
NP_EKF_pn1 = ax13.plot(range(len(NP_EKF_pn_data)),NP_EKF_pn_data, 'k')

ax14 = fig2.add_subplot(3, 3, 2)
ax14.set_xlabel('Time Step'), 
ax14.set_ylabel('Pe')
ax14.set_title('Error for Pe')
P_EKF_pe1 = ax14.plot(range(len(P_EKF_pe_data)),P_EKF_pe_data, 'b')
error_pe1 = ax14.plot(range(len(error_pe_hat)),error_pe_hat, 'r')
NP_EKF_pe1 = ax14.plot(range(len(NP_EKF_pe_data)),NP_EKF_pe_data, 'k')

ax15 = fig2.add_subplot(3, 3, 3)
ax15.set_xlabel('Time Step'), 
ax15.set_ylabel('Pd')
ax15.set_title('Error for Pd')
P_EKF_pd1 = ax15.plot(range(len(P_EKF_pd_data)),P_EKF_pd_data, 'b')
error_pd1 = ax15.plot(range(len(error_pd_hat)),error_pd_hat, 'r')
NP_EKF_pd1 = ax15.plot(range(len(NP_EKF_pd_data)),NP_EKF_pd_data, 'k')

ax16 = fig2.add_subplot(3, 3, 4)
ax16.set_xlabel('Time Step'), 
ax16.set_ylabel('u')
ax16.set_title('Error for u')
P_EKF_u1 = ax16.plot(range(len(P_EKF_u_data)),P_EKF_u_data, 'b')
error_u1 = ax16.plot(range(len(error_u_hat)),error_u_hat, 'r')
NP_EKF_u1 = ax16.plot(range(len(NP_EKF_u_data)),NP_EKF_u_data, 'k')

ax17 = fig2.add_subplot(3, 3, 5)
ax17.set_xlabel('Time Step'), 
ax17.set_ylabel('v')
ax17.set_title('Error for v')
P_EKF_v1 = ax17.plot(range(len(P_EKF_v_data)),P_EKF_v_data, 'b')
error_v1 = ax17.plot(range(len(error_v_hat)),error_v_hat, 'r')
NP_EKF_v1 = ax17.plot(range(len(NP_EKF_v_data)),NP_EKF_v_data, 'k')

ax18 = fig2.add_subplot(3, 3, 6)
ax18.set_xlabel('Time Step'), 
ax18.set_ylabel('w')
ax18.set_title('Error for w')
P_EKF_w1 = ax18.plot(range(len(P_EKF_w_data)),P_EKF_w_data, 'b')
error_w1 = ax18.plot(range(len(error_w_hat)),error_w_hat, 'r')
NP_EKF_w1 = ax18.plot(range(len(NP_EKF_w_data)),NP_EKF_w_data, 'k')

ax19 = fig2.add_subplot(3, 3, 7)
ax19.set_xlabel('Time Step'), 
ax19.set_ylabel('Psi')
ax19.set_title('Error for Psi')
P_EKF_psi1 = ax19.plot(range(len(P_EKF_psi_data)),P_EKF_psi_data, 'b')
error_psi1 = ax19.plot(range(len(error_psi_hat)),error_psi_hat, 'r')
NP_EKF_psi1 = ax19.plot(range(len(NP_EKF_psi_data)),NP_EKF_psi_data, 'k')